In [160]:
from genDistribution import genDist
import numpy as np 
import pandas as pd 
from collections import OrderedDict
import dill
from mylike_moped import kids450
import matplotlib.pylab as plt
import scipy.stats as ss

plt.rc('text', usetex=True)
plt.rc('font',**{'family':'sans-serif','serif':['Palatino']})
figSize  = (12, 8)
fontSize = 20

In [30]:
prior                 = OrderedDict()
prior['omega_cdm']    = [0.010, 0.390, 'uniform']
prior['omega_b']      = [0.019, 0.007, 'uniform']
prior['ln10^{10}A_s'] = [1.700, 3.300, 'uniform']
prior['n_s']          = [0.700, 0.600, 'uniform']
prior['h']            = [0.640, 0.180, 'uniform']
prior['A_bary']       = [0.000, 2.00, 'uniform']

prior['m_ncdm'] = [0.06, 0.94, 'uniform']
prior['A_IA']  = [-6.00, 12.00, 'uniform']

In [31]:
pdfs = genDist(prior)

# Generate a Test Point

In [32]:
def testpoint(Nsample = 1):
    rec = []
    for i in range(Nsample):
        rec.append([pdfs[j].rvs() for j in range(8)])
    return np.array(rec)

# Load GPs

In [25]:
gps = []
for i in range(11):
    with open('../semi-GPs/gp_1000/gp_'+str(i)+'.pkl', 'rb') as g:
        gps.append(dill.load(g))

# Predictions 

In [51]:
directory = '/home/harry/Desktop/kids-paper-2/'
kids = kids450(directory = directory, file_settings = 'scripts/settingsMoped', order = 'first', gpVar = True, includeGP = True, approximate = True)
kids.priors()
kids.loadMopedVectors(fileName = '2.txt')

Integrations performed at resolution of histogram!


In [107]:
def allPredictions(n):
    allRecords = []
    for i in range(n):
        records = np.zeros((11, 3))
        # testPoint for the emulator 
        testPoint      = testpoint(Nsample=1).flatten()
        
        #  KiDS-450 takes 11 inputs 
        testPoint_kids = np.concatenate([testPoint[0:6], np.zeros(3), [testPoint[-1]], [testPoint[-2]]], axis = 0)
        mopedCoeffs    = kids.mopedCoeff(testPoint_kids)
        records[:,0]   = mopedCoeffs
        
        for i in range(11):
            a, b = gps[i].prediction(testPoint, returnVar = True)
            records[i, 1] = a.flatten()
            records[i, 2] = np.sqrt(b.flatten())
            
        allRecords.append(records)
    
    return allRecords        

In [170]:
allPred = allPredictions(500)

In [197]:
# np.savez_compressed(directory + 'Data/allPreditions.npz', allPred)